## Error Analysis

In [1]:
import pandas as pd

In [54]:
df_svm = pd.read_csv('/Users/liki/neuefische/Respiratory_Disease_Classification/data/misclassified_svm.csv')
df_resnet = pd.read_csv('/Users/liki/neuefische/Respiratory_Disease_Classification/data/misclassified_testset_ResNet.csv')

In [57]:
# df_svm['filename'] = df_svm['filename'].apply(lambda x: x[2:6])
df_svm['filename'] = df_svm['filename'].apply(lambda x: x.replace('_',''))

In [66]:
file_svm = df_svm['filename'].unique()
file_svm.shape

(34,)

In [65]:
# df_resnet['filename'] = df_resnet['filename'].apply(lambda x: x[4:8])
df_resnet.tail(20)
file_rn = df_resnet['filename'].unique()
file_rn.shape

(42,)

In [68]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

print(len(intersection(file_rn, file_svm)))
intersection(file_rn, file_svm)

21


['f101',
 'i109',
 'i150',
 'i160',
 'i185',
 'i195',
 'i211',
 'i223',
 'i102',
 'i126',
 'i136',
 'i143',
 'i144',
 'i171',
 'i182',
 'i187',
 'i194',
 'i202',
 'i214',
 'i224',
 'i225']